In [25]:
import numpy as np
import time
import math
from scipy.interpolate import CubicSpline # Pour l'interpolation Spline
from scipy.integrate import quad # Pour l'intégration numérique précise (pour tester Lagrange)

In [26]:
def rectangle_midpoint(f, a, b, n):
    if n <= 0: raise ValueError("n doit être positif.")
    h = (b - a) / n
    integral = sum(f(a + (i + 0.5) * h) for i in range(n)) * h
    return integral

def trapezoid(f, a, b, n):
    if n <= 0: raise ValueError("n doit être positif.")
    h = (b - a) / n
    integral = (f(a) + f(b)) / 2.0 + sum(f(a + i * h) for i in range(1, n))
    return integral * h

def simpson(f, a, b, n):
    if n <= 0: raise ValueError("n doit être positif.")
    if n % 2 != 0: raise ValueError("n pour Simpson doit être pair.")
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n):
        x_i = a + i * h
        integral += 4 * f(x_i) if i % 2 == 1 else 2 * f(x_i)
    return (h / 3) * integral

In [27]:
def lagrange_basis_polynomial(x_points, j, x):
    """
    Calcule le j-ième polynôme de base de Lagrange L_j(x).
    
    Args:
        x_points (list or np.array): Les coordonnées x des points d'interpolation.
        j (int): L'indice du polynôme de base à calculer.
        x (float): Le point où évaluer L_j(x).
        
    Returns:
        float: La valeur de L_j(x).
    """
    numerator = 1.0
    denominator = 1.0
    
    for k in range(len(x_points)):
        if k == j:
            continue
        numerator *= (x - x_points[k])
        denominator *= (x_points[j] - x_points[k])
        
    return numerator / denominator

In [28]:
def lagrange_interpolating_polynomial(x_points, y_points):
    """
    Construit et retourne une fonction qui est le polynôme d'interpolation de Lagrange.
    
    Args:
        x_points (list or np.array): Les coordonnées x des points d'interpolation.
        y_points (list or np.array): Les coordonnées y (f(x)) des points d'interpolation.
        
    Returns:
        function: Une fonction P(x) qui évalue le polynôme de Lagrange à x.
    """
    if len(x_points) != len(y_points):
        raise ValueError("Les listes x_points et y_points doivent avoir la même longueur.")

    def P(x):
        result = 0.0
        for j in range(len(x_points)):
            result += y_points[j] * lagrange_basis_polynomial(x_points, j, x)
        return result
    return P

In [29]:
# --- Méthode de Simpson via Lagrange ---

def simpson_lagrange_analytic(f, a, b, n):
    """
    Calcule l'intégrale de f(x) de a à b en utilisant la méthode de Simpson,
    en construisant le polynôme de Lagrange de degré 2 sur chaque paire d'intervalles
    et en l'intégrant.

    Note: Pour cette implémentation, nous allons utiliser les coefficients (h/3, 4h/3, h/3)
    qui sont la conséquence directe de l'intégration analytique des polynômes de base de Lagrange.
    C'est la définition "analytique" de Simpson.
    
    Args:
        f (function): La fonction à intégrer.
        a (float): La borne inférieure de l'intervalle.
        b (float): La borne supérieure de l'intervalle.
        n (int): Le nombre de sous-intervalles (doit être pair).
        
    Returns:
        float: L'approximation de l'intégrale.
    """
    if n <= 0: raise ValueError("n doit être positif.")
    if n % 2 != 0: raise ValueError("n pour Simpson doit être pair.")

    h = (b - a) / n
    integral = 0.0

    # On parcourt les intervalles par paires (de x_0 à x_2, de x_2 à x_4, etc.)
    # Chaque segment de 2h est un "sous-intervalle" pour la formule de Simpson simple
    for i in range(0, n, 2): # i prend les valeurs 0, 2, 4, ..., n-2
        x0 = a + i * h
        x1 = a + (i + 1) * h
        x2 = a + (i + 2) * h # C'est le point x_{i+2}
        
        # Les coefficients de Simpson sont dérivés de l'intégration analytique
        # des polynômes de Lagrange de degré 2.
        # Nous appliquons directement ces coefficients.
        integral += (h / 3) * (f(x0) + 4 * f(x1) + f(x2))
        
    return integral

In [30]:
# --- Intégration via Spline Cubique Naturelle ---

def simpson_via_spline_integration(f, a, b, n):
    """
    Calcule l'intégrale de f(x) de a à b en utilisant une interpolation par Spline Cubique Naturelle
    qui passe par les points (x_i, f(x_i)), puis intègre cette Spline.
    Ce n'est pas la méthode de Simpson à proprement parler, mais une intégration d'une fonction interpolée.
    
    Args:
        f (function): La fonction à interpoler et intégrer.
        a (float): La borne inférieure de l'intervalle.
        b (float): La borne supérieure de l'intervalle.
        n (int): Le nombre de points à utiliser pour l'interpolation (nombre de sous-intervalles).
                 Pour n sous-intervalles, il y a n+1 points.
        
    Returns:
        float: L'approximation de l'intégrale de la spline.
    """
    if n <= 0: raise ValueError("n doit être positif.")
    
    # Générer les points d'interpolation
    x_points = np.linspace(a, b, n + 1) # n sous-intervalles => n+1 points
    y_points = f(x_points) # Valeurs de la fonction aux points

    # Créer la Spline Cubique Naturelle
    # bc_type='natural' signifie que la dérivée seconde aux extrémités est zéro.
    spline = CubicSpline(x_points, y_points, bc_type='natural')

    # Intégrer la spline sur l'intervalle [a, b]
    # La méthode .integrate(xa, xb) de CubicSpline intègre la spline analytiquement.
    integral_spline = spline.integrate(a, b)
    
    return integral_spline

In [31]:
# Fonction à intégrer
def f_x_squared(x):
    return x**2

def f_sin(x):
    return np.sin(x)

if __name__ == "__main__":
    a = 0
    b = 1
    n = 100 # Nombre de sous-intervalles (doit être pair pour Simpson)
    
    # Test des méthodes classiques
    print(f"--- Test des méthodes d'intégration classiques ---")
    print(f"Intégration de f(x) = x^2 sur [{a}, {b}] avec n={n}")
    print(f"Intégrale exacte : 1/3 = {1/3:.6f}\n")

    rect_result = rectangle_midpoint(f_x_squared, a, b, n)
    print(f"Méthode des rectangles (point milieu): {rect_result:.6f}")
    print(f"Erreur : {abs(rect_result - (1/3)):.6f}\n")

    trap_result = trapezoid(f_x_squared, a, b, n)
    print(f"Méthode des trapèzes: {trap_result:.6f}")
    print(f"Erreur : {abs(trap_result - (1/3)):.6f}\n")

    simpson_result = simpson(f_x_squared, a, b, n)
    print(f"Méthode de Simpson: {simpson_result:.6f}")
    print(f"Erreur : {abs(simpson_result - (1/3)):.6f}\n")

    print(f"\n--- Test de Simpson via Lagrange et Spline ---")

    # 1. Simpson via l'approche analytique de Lagrange (même que simpson)
    simpson_lagrange_result = simpson_lagrange_analytic(f_x_squared, a, b, n)
    print(f"Méthode de Simpson (Lagrange analytique): {simpson_lagrange_result:.6f}")
    print(f"Erreur : {abs(simpson_lagrange_result - (1/3)):.6f}\n")
    # Remarque: Ce sera le même résultat que simpson_result car c'est la même formule.

    # 2. Intégration de Spline Cubique
    # Ici, 'n' est le nombre de sous-intervalles pour la spline, donc n+1 points.
    # Pour avoir une comparaison équitable, utilisons le même 'n' que pour les autres méthodes.
    spline_result = simpson_via_spline_integration(f_x_squared, a, b, n)
    print(f"Intégration via Spline Cubique Naturelle: {spline_result:.6f}")
    print(f"Erreur : {abs(spline_result - (1/3)):.6f}\n")
    # Pour f(x)=x^2, la spline cubique naturelle passera par les points.
    # Une fonction quadratique est un cas particulier d'un polynôme cubique si le coefficient x^3 est nul.
    # Une spline cubique naturelle peut donc interpoler x^2 exactement si les points sont bien choisis ou si les conditions aux bords sont satisfaites.
    # Pour un polynôme de degré 2, la spline cubique peut le reproduire si les conditions aux limites naturelles sont appropriées.
    # Pour sin(x), les erreurs seront plus visibles.

    print("-" * 30)

    a_sin = 0
    b_sin = math.pi
    n_sin = 1000 # Doit être pair pour Simpson
    exact_sin_integral = 2.0 

    print(f"--- Test avec f(x) = sin(x) sur [{a_sin:.2f}, {b_sin:.2f}] avec n={n_sin} ---")
    print(f"Intégrale exacte : {exact_sin_integral:.6f}\n")

    simpson_lagrange_sin_result = simpson_lagrange_analytic(f_sin, a_sin, b_sin, n_sin)
    print(f"Méthode de Simpson (Lagrange analytique): {simpson_lagrange_sin_result:.6f}")
    print(f"Erreur : {abs(simpson_lagrange_sin_result - exact_sin_integral):.6f}\n")

    spline_sin_result = simpson_via_spline_integration(f_sin, a_sin, b_sin, n_sin)
    print(f"Intégration via Spline Cubique Naturelle: {spline_sin_result:.6f}")
    print(f"Erreur : {abs(spline_sin_result - exact_sin_integral):.6f}\n")

    # Exemple d'utilisation de lagrange_interpolating_polynomial pour visualisation (non pour l'intégration directe ici)
    x_test_points = np.linspace(a, b, 3)
    y_test_points = f_x_squared(x_test_points)
    P_lagrange = lagrange_interpolating_polynomial(x_test_points, y_test_points)
    
    print("\n--- Exemple d'évaluation d'un polynôme de Lagrange (3 points) ---")
    print(f"Polynôme de Lagrange pour f(x)=x^2 passant par {x_test_points}:")
    print(f"P(0.5) = {P_lagrange(0.5):.4f} (f(0.5) = {f_x_squared(0.5):.4f})")
    print(f"P(0.2) = {P_lagrange(0.2):.4f} (f(0.2) = {f_x_squared(0.2):.4f})")

--- Test des méthodes d'intégration classiques ---
Intégration de f(x) = x^2 sur [0, 1] avec n=100
Intégrale exacte : 1/3 = 0.333333

Méthode des rectangles (point milieu): 0.333325
Erreur : 0.000008

Méthode des trapèzes: 0.333350
Erreur : 0.000017

Méthode de Simpson: 0.333333
Erreur : 0.000000


--- Test de Simpson via Lagrange et Spline ---
Méthode de Simpson (Lagrange analytique): 0.333333
Erreur : 0.000000

Intégration via Spline Cubique Naturelle: 0.333333
Erreur : 0.000000

------------------------------
--- Test avec f(x) = sin(x) sur [0.00, 3.14] avec n=1000 ---
Intégrale exacte : 2.000000

Méthode de Simpson (Lagrange analytique): 2.000000
Erreur : 0.000000

Intégration via Spline Cubique Naturelle: 2.000000
Erreur : 0.000000


--- Exemple d'évaluation d'un polynôme de Lagrange (3 points) ---
Polynôme de Lagrange pour f(x)=x^2 passant par [0.  0.5 1. ]:
P(0.5) = 0.2500 (f(0.5) = 0.2500)
P(0.2) = 0.0400 (f(0.2) = 0.0400)
